# Full data experiments

In [ ]:
from prutils import *
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

datasets = [RAW_PIXELS_DATASET, IM_FEATURES_DATASET]

for dataset in datasets:
    X, y = get_full_data(dataset, split_validation=False)

    """
    Results without processing
    """
    lda = LDA().fit(X, y)
    performance = estimate_classifier_performance_normal(lda, RAW_PIXELS_TEST)
    print "{} - feature reduction: {}%".format(dataset, performance)

    """
    Results with PCA
    """
    pca = PCA(n_components=30)
    lda = LDA().fit(pca.fit_transform(X), y)
    performance = estimate_classifier_performance_transform(lda, RAW_PIXELS_TEST, pca)
    print "{} - PCA: {}%".format(dataset, performance)

    """
    Results with ICA
    """
    ica = FastICA(n_components=30)
    lda = LDA().fit(ica.fit_transform(X), y)
    performance = estimate_classifier_performance_transform(lda, RAW_PIXELS_TEST, ica)
    print "{} - ICA: {}%".format(dataset, performance)

# Batched data experiments

In [ ]:
from prutils import *
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA 
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


"""
Results without processing
"""
X, y = get_random_batch(RAW_PIXELS_DATASET, split_validation=False)
lda = LDA().fit(X, y)
print "Performance without feature reduction: {}%".format(estimate_classifier_performance_normal(lda, RAW_PIXELS_TEST))

"""
Results with PCA
"""
X, y = get_random_batch(RAW_PIXELS_DATASET, split_validation=False)
pca = PCA(n_components=20)
lda = LDA()
lda.fit(pca.fit_transform(X), y)
print "Performance with PCA: {}%".format(estimate_classifier_performance_transform(lda, RAW_PIXELS_TEST, pca))

"""
Results with PCA
"""
X, y = get_random_batch(RAW_PIXELS_DATASET, split_validation=False)
ica = FastICA(n_components=20)
lda = LDA()
lda.fit(ica.fit_transform(X), y)
print "Performance with ICA: {}%".format(estimate_classifier_performance_transform(lda, RAW_PIXELS_TEST, ica))

### PCA experiments

In [1]:
from prutils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA 
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

pca_full_raw = run_PCA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50, batch=False)
pca_batch_raw = run_PCA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50, batch=True)

pca_full_features = run_PCA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50, batch=False)
pca_batch_features = run_PCA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50, batch=True)

plot_performance([pca_full_raw, pca_batch_raw, pca_full_features, pca_batch_features], show_results=False, save_to_file="lda_pca")

### ICA experiments

In [1]:
from prutils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

ica_full_raw = run_ICA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50, batch=False)
ica_batch_raw = run_ICA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50, batch=True)

ica_full_features = run_ICA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50, batch=False)
ica_batch_features = run_ICA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50, batch=True)

plot_performance([ica_full_raw, ica_batch_raw, ica_full_features, ica_batch_features], show_results=False, save_to_file="lda_ica")

### Kernel PCA experiments

In [2]:
from prutils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


kpca_full_raw = run_KPCA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50)
kpca_batch_raw = run_KPCA_experiment(LDA(), RAW_PIXELS_DATASET, max_components = 50)

kpca_full_features = run_KPCA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50)
kpca_batch_features = run_KPCA_experiment(LDA(), IM_FEATURES_DATASET, max_components = 50)

plot_performance([kpca_full_raw, kpca_batch_raw, kpca_full_features, kpca_batch_features],
                 show_results=False, save_to_file="lda_kpca")

KeyboardInterrupt: 